In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import DataGenerator
from data_agent import *
from stock_agent import *
from backtest import *
from bitstamp import *

from sklearn.utils import shuffle

import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from keras.wrappers.scikit_learn import KerasClassifier
from imblearn.over_sampling import RandomOverSampler
import random
from catboost import CatBoost

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.utils.validation import check_is_fitted

from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import *
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE, RFECV

path = "./data/"


In [2]:
#train_start_list = [1622502000, 1590966000, 1559343600, 1580515200, 1612137600]
train_start_list = [1622502000, 1590966000, 1559343600, 1527807600, 1517443200, 1548979200, 1580515200, 1612137600]
#train_start_list = [1612137600]
train_keys = ["btcusd", "ethusd", "bchbtc"]
#train_keys = ["bchbtc"]
val_start = 1626340500
val_end = 1626369600

In [3]:
load_from_disk = False

# '2Min'

In [4]:
def get_online_data(minutes, sourceDataGenerator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 sourceDataGenerator = sourceDataGenerator,
                 minutes = minutes,
                 train_keys = train_keys,
                 train_limit = 400,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = val_start,
                 val_end = val_end,
                 train_start_list = train_start_list
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [5]:

def getModel():
    param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
    #param = {'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 3, 'random_state': 42}
    coModel = RandomForestClassifier(**param)
    return coModel

In [6]:
%%time

def get_classifiers():
    clss = classifiers = [
        lambda : MLPClassifier(alpha=1, max_iter=1000, random_state = 42),
        
        #lambda : GaussianNB(),
        #lambda : QuadraticDiscriminantAnalysis(),
        #lambda : AdaBoostClassifier(random_state = 42),
        #lambda : CatBoostClassifier(logging_level = 'Silent'),
        
        
        #lambda : LogisticRegression(),
        #KNeighborsClassifier(3),
        #GaussianProcessClassifier(1.0 * RBF(1.0)),
        
        
        #lambda : DecisionTreeClassifier(random_state = 42),
        #lambda : RandomForestClassifier(max_depth=50, n_estimators=100, max_features=1),
        
        #lambda : SVC(gamma=2, C=1, random_state = 42),
    ]
    
    def rand():
        ##print("rand created")
        param = {'criterion': 'entropy', 'max_depth': 100, 'n_estimators': 30, 'random_state': 42}
        return RandomForestClassifier(**param)
    

    #clss = []
    #clss.append(lambda : rand())
    return clss


estimators = [
                #RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=6),
                lambda : RFE(estimator=RandomForestClassifier(random_state=10), n_features_to_select=10),
                lambda : RFE(estimator=DecisionTreeClassifier(random_state=10), n_features_to_select=8),
                lambda : RFECV(estimator=DecisionTreeClassifier(random_state=10)),
                lambda : RFECV(estimator=RandomForestClassifier(random_state=10))
                ]


def get_all_models():
    models = []
    classifiers = get_classifiers()
    for cls in classifiers:
        models.append(cls())
        #models.append(MockModel(cls()))
        #models.append(MockCoPilotModel(cls(), getModel()))
        
        for est in estimators:
            tmp = lambda : Pipeline(
                steps=[
                ('s',est()),
                ('m',cls())
            ])
            models.append(tmp())
            #models.append(MockModel(tmp()))
            #models.append(MockCoPilotModel(tmp(), getModel()))
    return models


CPU times: user 10 µs, sys: 8 µs, total: 18 µs
Wall time: 43.2 µs


In [7]:
get_all_models_factory = lambda : get_all_models()

In [8]:
%%time

def test_models(provider, get_all_models_factory, steps = 5):
    
    score_board_by_step = {}

    for step in tqdm(range(1, steps + 1)):
        print(f"Step {step}")
        models_list = get_all_models_factory()
        model_result = []
        for idx in tqdm(range(len(models_list))):
            model_creator = models_list[idx]
            model = model_creator
            #print(f"Trainning {model}")
            result = run_trial(model_creator, provider, steps)

            result['step'] = step

            model_result.append(result)
        
        score_board_by_step[step] = model_result
        
    best_models = []

    best_model = []

    best_score = -1
    
    best_step = -1
    
    model_rank = []
    
    def myFunc(e):
        return e['profit']

    for step_idx in score_board_by_step:
        step_board = score_board_by_step[step_idx]
        for result in step_board:
            if (result['profit'] > 0):
                model_rank.append(result)
            
    model_rank.sort(key=myFunc, reverse = True)
    
    return model_rank

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.82 µs


In [9]:

steps = 1

windows = [10, 20, 30, 40]


In [10]:

def test_windows(minutes):
    results = []
    for win in tqdm(windows):
        tec = TecAn(windows = win, windows_limit = 100)
        sourceDataGenerator = SourceDataGenerator(tec = tec)
        online = get_online_data(minutes, sourceDataGenerator, load_from_disk, win)
        result = test_models(
            online, 
            get_all_models_factory,
            steps = 1
        )
        results.append(result)
    return results
    

In [ ]:
%%time
results_15 = test_windows(15)

  0%|          | 0/4 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/btcusd/ -> {'currency_pair': 'btcusd', 'step': 900, 'limit': 400, 'start': 1622502000}
Downloaded from 2021-06-01 00:00:00 to 2021-06-05 03:45:00 37477.81
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 399 of btcusd


  0%|          | 0/399 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/ethusd/ -> {'currency_pair': 'ethusd', 'step': 900, 'limit': 400, 'start': 1622502000}
Downloaded from 2021-06-01 00:00:00 to 2021-06-05 03:45:00 2749.93
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 399 of ethusd


  0%|          | 0/399 [00:00<?, ?it/s]

...

  0%|          | 0/399 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/ethusd/ -> {'currency_pair': 'ethusd', 'step': 900, 'limit': 400, 'start': 1517443200}
Downloaded from 2018-02-01 00:00:00 to 2018-02-05 03:45:00 837.40
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 399 of ethusd


  0%|          | 0/399 [00:00<?, ?it/s]

https://www.bitstamp.net/api/v2/ohlc/bchbtc/ -> {'currency_pair': 'bchbtc', 'step': 900, 'limit': 400, 'start': 1517443200}
Downloaded from 2018-02-01 00:00:00 to 2018-02-05 03:45:00 0.13851397
Resample 15Min - TecAn ( windows 10, windows_limit 100 )
Processing 399 of bchbtc


  0%|          | 0/399 [00:00<?, ?it/s]

In [ ]:
for r in results_15:
    best_result = r[0]
    best_step_180 = best_result['step']
    best_score_180 = best_result['profit']
    best_model_180 = best_result['model']
    #print(result)
    print(f"{best_step_180}: {best_score_180}")
    print(best_model_180) #0.7366


In [ ]:
results_5 = test_windows(5)

In [ ]:
for r in results_5:
    if (len(r) > 0):
        best_result = r[0]
        best_step_180 = best_result['step']
        best_score_180 = best_result['profit']
        best_model_180 = best_result['model']
        #print(result)
        print(f"{best_step_180}: {best_score_180}")
        print(best_model_180)
    else:
        print("Not found")

In [ ]:
results_3 = test_windows(3)

In [ ]:
for r in results_3:
    if (len(r) > 0):
        best_result = r[0]
        best_step_180 = best_result['step']
        best_score_180 = best_result['profit']
        best_model_180 = best_result['model']
        #print(result)
        print(f"{best_step_180}: {best_score_180}")
        print(best_model_180)
    else:
        print("Not found")

In [ ]:
results_1 = test_windows(1)

In [ ]:
for r in results_1:
    if (len(r) > 0):
        best_result = r[0]
        best_step_180 = best_result['step']
        best_score_180 = best_result['profit']
        best_model_180 = best_result['model']
        #print(result)
        print(f"{best_step_180}: {best_score_180}")
        print(best_model_180)
    else:
        print("Not found")

In [ ]:
def eval_model(model, provider, step):
    
    #train_by_step(model, step, provider)
    
    print("Reference")
    for train_set in provider.val_keys:
        trainX_raw, trainY_raw = provider.load_val_data(train_set)
        x, y, closed_prices = get_sequencial_data(trainX_raw, trainY_raw, step)
        reference = get_max_profit(x, y, closed_prices, step)
        print(f"{train_set} Profit: {reference.get_profit()}%")
    print("")
    print("Model")
    profits = []
    for train_set in provider.val_keys:
        print(f"{train_set}")

        valX, valY = provider.load_val_data(train_set)

        x, y, closed_prices = get_sequencial_data(valX, valY, step)

        #print(np.array(x).shape)

        preds = model.predict(x)

        recall = recall_score(y, preds)
        precision = precision_score(y, preds)
        f1 = f1_score(y, preds)
        accuracy = accuracy_score(y, preds)

        back = BackTest(value = 100, 
                        verbose = False, 
                        sell_on_profit = True,
                        pending_sell_steps = step)
        back = backtest_model(model, x, closed_prices, back)

        score = (precision, recall, f1, accuracy)

        print(f"{back.get_profit()}")
        print(f"{score}")
        print(f"{back.get_profit() / reference.get_profit()}")
        print(f"")

        profits.append(back.get_profit())

    print(np.average(profits))

In [ ]:
eval_model(best_model_60, online60, best_step_60)

In [ ]:
eval_model(best_model_180, online180, best_step_180)

In [ ]:
online = OnLineDataProvider(
             step = 300,
             train_keys = ["btcusd"],
             train_limit = 10,
             val_limit = 1000,
             val_keys = ["btcusd"],
             val_start = 1626131509,
             train_start_list = [train_start_list[-1]]
)

online.load_cache()


eval_model(best_model_60, online, best_step_60)

In [ ]:
online = OnLineDataProvider(
             step = 180,
             train_keys = ["btcusd"],
             train_limit = 10,
             val_limit = 1000,
             val_keys = ["btcusd"],
             val_start = 1625872309,
             train_start_list = [train_start_list[-1]]
)

online.load_cache()

eval_model(best_model_180, online, best_step_60)

In [ ]:
def test_model(model, set_key, provider, step):
    valX, valY = provider.load_val_data(set_key)

    x, y, closed_prices = get_sequencial_data(valX, valY, step)
    
    print(len(x))

    preds = model.predict(x)

    recall = recall_score(y, preds)
    precision = precision_score(y, preds)
    f1 = f1_score(y, preds)
    accuracy = accuracy_score(y, preds)

    back = BackTest(value = 100, 
                    verbose = True, 
                    sell_on_profit = True,
                    pending_sell_steps = step)
    back = backtest_model(model, x, closed_prices, back)
    return back

In [ ]:

back = test_model(best_model_60, "btcusd", online, 1)

In [ ]:
#back.request_sell(34123.38)
back.get_profit()